# Practical Exam: House sales

RealAgents is a real estate company that focuses on selling houses.

RealAgents sells a variety of types of house in one metropolitan area.

Some houses sell slowly and sometimes require lowering the price in order to find a buyer.

In order to stay competitive, RealAgents would like to optimize the listing prices of the houses it is trying to sell.

They want to do this by predicting the sale price of a house given its characteristics.

If they can predict the sale price in advance, they can decrease the time to sale.


## Data

The dataset contains records of previous houses sold in the area.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton'. </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced" (two shared walls), "Semi-detached" (one shared wall), or "Detached" (no shared walls). </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |


# Task 1

The team at RealAgents knows that the city that a property is located in makes a difference to the sale price. 

Unfortuntately they believe that this isn't always recorded in the data. 

Calculate the number of missing values of the `city`. 

 - You should use the data in the file "house_sales.csv". 

 - Your output should be an object `missing_city`, that contains the number of missing values in this column. 

In [174]:
# Use this cell to write your code for Task 1
import pandas as pd
import numpy as np

house_sales=pd.read_csv('house_sales.csv')

missing_city = (house_sales['city'] == '--').sum()
missing_city

73

# Task 2 

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "house_sales.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.


| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton' </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced", "Semi-detached", or "Detached". </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |

In [175]:
# Use this cell to write your code for Task 2
house_sales['city']=house_sales['city'].str.replace('--','Unknown')
house_sales['city']=house_sales['city'].astype('category')

house_sales['months_listed']=house_sales['months_listed'].fillna(round(house_sales['months_listed'].mean(),1))

house_sales['sale_date']=pd.to_datetime(house_sales['sale_date'])

house_sales['house_type']=house_sales['house_type'].replace({'Det.':'Detached',
                                                   'Semi':'Semi-detached',
                                                   'Terr.':'Terraced'})

cat_order=pd.CategoricalDtype(['Terraced','Semi-detached','Detached'],
                                           ordered=True)
house_sales['house_type']=house_sales['house_type'].astype(cat_order)

house_sales['area']=house_sales['area'].astype('string')
house_sales['area']=house_sales['area'].str.replace('sq.m.','')
house_sales['area']=house_sales['area'].astype('float')

clean_data = house_sales.copy()

# Task 3 

The team at RealAgents have told you that they have always believed that the number of bedrooms is the biggest driver of house price. 

Producing a table showing the difference in the average sale price by number of bedrooms along with the variance to investigate this question for the team.

 - You should start with the data in the file 'house_sales.csv'.

 - Your output should be a data frame named `price_by_rooms`. 

 - It should include the three columns `bedrooms`, `avg_price`, `var_price`. 

 - Your answers should be rounded to 1 decimal place.   

In [176]:
# Use this cell to write your code for Task 3
price_by_rooms = house_sales.groupby('bedrooms').agg(avg_price=('sale_price','mean'),
                                    var_price=('sale_price','var'))
price_by_rooms['avg_price'] = np.round(price_by_rooms['avg_price'],1)
price_by_rooms['var_price'] = np.round(price_by_rooms['var_price'],1)

price_by_rooms=price_by_rooms.reset_index().reset_index(drop=True)

print(price_by_rooms)

   bedrooms  avg_price     var_price
0         2    67076.4  5.652896e+08
1         3   154665.1  2.378289e+09
2         4   234704.6  1.725211e+09
3         5   301515.9  2.484328e+09
4         6   375741.3  3.924432e+09


# Task 4

Fit a baseline model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [177]:
# Use this cell to write your code for Task 4
from sklearn.linear_model import LinearRegression

train = pd.read_csv('train.csv')
validation=pd.read_csv('validation.csv')

dummies01 = pd.get_dummies(train['city'])
dummies02 = pd.get_dummies(train['house_type'])
train = pd.concat([train,dummies01,dummies02],axis=1)
train = train.drop(['city','house_type'],axis=1)

dummies01 = pd.get_dummies(validation['city'])
dummies02 = pd.get_dummies(validation['house_type'])
validation = pd.concat([validation,dummies01,dummies02],axis=1)
validation = validation.drop(['city','house_type'],axis=1)

X_train = train.drop(['sale_price','sale_date'],axis=1)
y_train = train['sale_price']
X_test = validation.drop('sale_date',axis=1)

model=LinearRegression()
model.fit(X_train,y_train)
data_pred=model.predict(X_test)

base_result = pd.DataFrame({'house_id':X_test['house_id'],
                            'price':data_pred})

# Task 5

Fit a comparison model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [178]:
# Use this cell to write your code for Task 5
from sklearn.ensemble import RandomForestRegressor

train = pd.read_csv('train.csv')
validation=pd.read_csv('validation.csv')

dummies01 = pd.get_dummies(train['city'])
dummies02 = pd.get_dummies(train['house_type'])
train = pd.concat([train,dummies01,dummies02],axis=1)
train = train.drop(['city','house_type'],axis=1)

dummies01 = pd.get_dummies(validation['city'])
dummies02 = pd.get_dummies(validation['house_type'])
validation = pd.concat([validation,dummies01,dummies02],axis=1)
validation = validation.drop(['city','house_type'],axis=1)

X_train = train.drop(['sale_price','sale_date'],axis=1)
y_train = train['sale_price']
X_test = validation.drop('sale_date',axis=1)

ranreg = RandomForestRegressor(n_estimators=100,min_samples_leaf=2,max_depth=20,random_state=42)
ranreg.fit(X_train,y_train)
data_pred=ranreg.predict(X_test)

compare_result = pd.DataFrame({'house_id':X_test['house_id'],
                               'price':data_pred})